

---


# **Introduction au Machine Learning avec Scikit-learn**
## **Partie II : Modèles simples de classification**


---

Pour cette seconde partie d'introduction au module `scikit-learn`, nous allons nous intéresser au deuxième type de problème en Machine Learning : le problème de **classification**.

L'objectif de cette introduction est :

- D'introduire le problème de classification.
- D'apprendre à utiliser le module `scikit-learn` pour construire un modèle de classification, aussi appelé «classifieur».
- D'introduire des métriques utiles à l'évaluation des performances du modèle.

## **Introduction à la classification**
### **Objectif de la classification**

En Apprentissage supervisé, l'objectif est de prédire la valeur d'une variable cible à partir de variables explicatives.

- Dans un problème de **régression**, la variable cible prend des **valeurs continues**. Ces valeurs sont numériques : prix d'une maison, quantité d'oxygène dans l'air d'une ville, etc...
La variable cible peut donc prendre une **infinité de valeurs.**

- Dans un problème de **classification**, la variable cible prend des **valeurs discrètes**. Ces valeurs peuvent être numériques ou littérales mais dans les deux cas, la variable cible prend un **nombre fini de valeurs**.
Les différentes valeurs prises par la variable cible sont ce qu'on appelle des **classes**.

**L'objectif de la classification consiste donc à prédire la classe d'une observation à partir de ses variables explicatives.**

### **Un exemple de classification**
Prenons un exemple de classification **binaire**, autrement dit où il y a **deux** classes.
Nous cherchons à déterminer si l'eau d'un ruisseau est potable ou non en fonction de sa concentration en substances toxiques et de sa teneur en sels minéraux.

Les deux classes sont donc **'potable'** et **'non potable'**.

![sklearn_intro_classification_binaire](https://github.com/diaBabPro/colabs/blob/main/sklearn_intro_classification_binaire.png?raw=true)

Sur la figure ci-dessus, chaque point représente un ruisseau dont la position sur le plan est définie par ses valeurs de concentration en substances toxiques et de teneur en sels minéraux.

L'objectif sera de construire un **modèle capable d'attribuer une des deux classes** ('potable'/'non potable') à un ruisseau dont on ne connait que ces deux variables.

La figure ci-dessus suggère l'existence de deux zones permettant de classifier les ruisseaux facilement :

- Une zone où les ruisseaux sont potables (en haut à gauche).
- Une zone où les ruisseaux sont non potables (en bas à droite).

Nous aimerions créer un modèle capable de **séparer le jeu de données en deux parties** correspondant à ces zones.

Une technique simple serait de séparer les deux zones à **l'aide d'une ligne**.

- **(a)** Exécuter la cellule suivante pour afficher la figure interactive.

> - Les points **oranges** sont les ruisseaux **potables** et les points **bleus** sont les ruisseaux **non-potables**.
> - La **flèche rouge** correspond à un **vecteur** défini par  $𝑤=(𝑤1,𝑤2)$
 . La ligne rouge correspond au plan orthogonal (i.e. perpendiculaire) à  𝑤
 . Vous pouvez modifier les coordonnées du vecteur  𝑤
  de deux façons :
    - En faisant défiler les curseurs `w_1` et `w_2`.
    - En cliquant sur les valeurs à droite des curseurs puis en insérant directement la valeur souhaitée.

- **(b)** Essayer de trouver un vecteur  𝑤
  tel que **le plan orthogonal à  𝑤
  sépare parfaitement les deux classes de ruisseau**.
- **(c)** Une solution possible est donnée par le vecteur  $𝑤=(−1.47,0.84)$
 . Est-ce que le vecteur  $𝑤=(1.47,−0.84)$
  donne aussi une solution ?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def linear_classification():
    np.random.seed(42)
    n_points = 100

    X_potable = np.random.multivariate_normal([-1, 1], [[0.2, 0], [0, 0.2]], n_points)

    X_non_potable = np.random.multivariate_normal([1, -1], [[0.2, 0], [0, 0.2]], n_points)

    X = np.vstack([X_potable, X_non_potable])
    y = np.hstack([np.ones(n_points), -1 * np.ones(n_points)])

    def plot_classification(w1, w2):
        plt.figure(figsize=(8, 6))

        plt.scatter(X_potable[:, 0], X_potable[:, 1], color='orange', label='Potable')
        plt.scatter(X_non_potable[:, 0], X_non_potable[:, 1], color='blue', label='Non-potable')

        w = np.array([w1, w2])
        slope = -w[0] / w[1]
        intercept = 0

        x_vals = np.linspace(-3, 3, 100)
        y_vals = slope * x_vals + intercept
        plt.plot(x_vals, y_vals, 'r', label=f'Séparation: w=({w1:.2f}, {w2:.2f})')

        plt.quiver(0, 0, w[0], w[1], angles='xy', scale_units='xy', scale=1, color='red')

        plt.xlim(-3, 3)
        plt.ylim(-3, 3)
        plt.axhline(0, color='black',linewidth=0.5)
        plt.axvline(0, color='black',linewidth=0.5)
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.legend(loc='upper right')
        plt.title('Classification des ruisseaux')
        plt.grid(True)
        plt.show()

    interact(plot_classification, w1=FloatSlider(min=-3, max=3, step=0.01, value=-1.47), w2=FloatSlider(min=-3, max=3, step=0.01, value=0.84))

linear_classification()

La classification que nous venons de faire est de type linéaire, c'est-à-dire que nous avons utilisé un plan linéaire pour séparer nos classes.

Ainsi, l'objectif des modèles de classification linéaires est de trouver le vecteur  𝑤
  permettant de séparer au mieux les différentes classes.
Chaque modèle de type linéaire dispose de sa propre technique pour trouver ce vecteur.

Il existe aussi des modèles de classification non-linéaires, que nous verrons plus tard.

![sklearn_intro_classification_lin_non_lin](https://github.com/diaBabPro/colabs/blob/main/sklearn_intro_classification_lin_non_lin.png?raw=true)


## **1.   Utilisation de `scikit-learn` pour la classification**


Nous allons maintenant introduire les principaux outils du module `scikit-learn` essentiels à la résolution d'un problème de classification.

Dans cet exercice, nous utiliserons le jeu de données [Congressional Voting Records](https://archive.ics.uci.edu/ml/datasets/congressional+voting+records) qui contient un nombre de votes faits par les membres du Congrès de la Chambre des Représentants des États-Unis.

L'objectif de notre problème de classification sera de **prédire le parti politique** ("démocrate" ou "républicain") des membres de la Chambre des Représentants en fonction de leurs votes sur des sujets comme l'éducation, la santé, le budget, etc...

Les variables explicatives seront donc les votes sur différents sujets et la variable cible sera le parti politique "démocrate" ou "républicain".

Pour résoudre ce problème nous allons utiliser un modèle de classification linéaire : la **Régression Logistique**.

### **Préparation des données**
- **(a)** Exécuter la cellule suivante pour importer les modules `pandas` et `numpy` nécessaires à la suite de l'exercice.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

- **(b)** Charger les données contenues dans le fichier `'votes.csv'` dans un `DataFrame` nommé votes.

In [ ]:
# Insérez votre code ici

Afin de visualiser brièvement nos données :

- **(c)** Afficher le nombre de lignes et de colonnes de `votes`.
- **(d)** Afficher un aperçu des 20 premières lignes de `votes`.

In [ ]:
# Insérez votre code ici

- La première colonne **`"party"`** contient le nom du **parti politique** auquel chaque membre du Congrès de la Chambre des Représentants appartient.
- Les **16 colonnes** suivantes contiennent les votes de chaque membre du Congrès sur des propositions de lois :
  - `'y'` indique que l'élu a voté **pour** la proposition de loi.
  - `'n'` indique que l'élu a voté **contre** la proposition de loi.
Afin d'utiliser les données dans un modèle de classification, il est nécessaire de transformer ces colonnes en valeurs **numériques** binaires, autrement dit soit 0 soit 1.

- **(e)** Pour chacune des colonnes 1 à 16 (la colonne 0 étant notre variable cible), remplacer les valeurs `'y'` par 1 et `'n'` par 0. Pour cela, on peut s'aider de la méthode **`replace`** de la classe `DataFrame`.
- **(f)** Afficher les 10 premières lignes du `DataFrame` modifié.

In [ ]:
# Insérez votre code ici

- **(g)** Dans un `DataFrame` nommé `X`, stocker les variables **explicatives** du jeu de données (toutes les colonnes sauf `'party'`). Pour cela, vous pourrez vous aider de la méthode **`drop`** d'un `DataFrame`.
- **(h)** Dans une `Series` nommé `y`, stocker la **variable cible** (`'party'`).

In [ ]:
# Insérez votre code ici

Comme pour la régression, nous allons devoir séparer le jeu de données en 2 parties : un jeu **d'entraînement** et un jeu de **test**. Pour rappel :

>- Le jeu d'entraînement sert à **entraîner le modèle** de classification, c'est-à-dire trouver les paramètres du modèle qui séparent au mieux les classes.
>- Le jeu de test sert à **évaluer** le modèle sur des données qu'il n'a jamais vues. Cette évaluation nous permettra de juger sur la capacité à **généraliser** du modèle.

- **(i)** Importer la fonction `train_test_split` du sous module `sklearn.model_selection`. On rappelle que cette fonction s'utilise ainsi :

```python
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
```

- **(j)** Séparer les données en un jeu d'entraînement `(X_train, y_train)` et un jeu de test `(X_test, y_test)` en gardant 20% des données pour l'échantillon de test.

>Pour éliminer l'aléa de la fonction `train_test_split`, vous pouvez utiliser le paramètre `random_state` avec une valeur entière (par exemple random_state = 2).
Ainsi, à chaque fois que vous utiliserez la fonction avec l'argument `random_state = 2`, les jeux de données produits seront les mêmes.

In [ ]:
# Insérez votre code ici

Le modèle de régression logistique est étroitement lié au modèle de **régression linéaire** vu dans le précédent notebook.

Il ne faut pas les **confondre** puisqu'ils ne résolvent pas les mêmes types de problèmes :

La régression **logistique** est utilisée pour la classification (prédire des classes).
La régression **linéaire** est utilisée pour la régression (prédire une variable quantitative).
Le modèle de régression linéaire était défini par la formule suivante :
$𝑦≈β0+\sum_{j=1}^{p} β𝑗𝑥𝑗$

La régression logistique n'estime plus  𝑦
  directement mais la **probabilité** que  𝑦
  soit égal à 0 ou 1.
Ainsi, le modèle est défini par la formule :

$𝑃(𝑦=1)=𝑓(𝛽0+\sum_{j=1}^{p}𝛽𝑗𝑥𝑗)$

Où

$f(x) = \frac{1}{1 + e^{-x}}$


La fonction  𝑓
 , souvent appelée **sigmoïde** ou **fonction logistique**, permet de transformer la combinaison linéaire  $𝑦≈β0+\sum_{j=1}^{p} β𝑗𝑥𝑗$
  en une valeur comprise entre 0 et 1 que l'on pourra interpréter comme une **probabilité** :

- Si  $𝑦≈β0+\sum_{j=1}^{p} β𝑗𝑥𝑗$
  est positif, alors  $𝑃(𝑦=1)>0.5$
 , donc la classe prédite de l'observation sera 1.

- Si  $𝑦≈β0+\sum_{j=1}^{p} β𝑗𝑥𝑗$
  est négatif, alors  $𝑃(𝑦=1)<0.5$
 , c'est-à-dire que  $𝑃(𝑦=0)>0.5$
 , donc la classe prédite de l'observation sera 0.

- **(k)** Importer la classe `LogisticRegression` du sous-module `linear_model` de `scikit-learn`.
- **(l)** Instancier un modèle `LogisticRegression` nommé **`logreg`** sans préciser d'arguments du constructeur.
- **(m)** Entraîner le modèle sur le jeu de données d'entraînement grâce à la méthode `fit` de la classe `LogisticRegression`.
- **(n)** Effectuer une prédiction sur les données de **test**. Stocker ces prédictions dans **`y_pred_test_logreg`** et afficher les 10 premières prédictions.

In [ ]:
# Insérez votre code ici

## **2. Evaluer la performance d'un modèle de classification**

Il existe différentes métriques pour évaluer les performances de modèles de classification comme :

- L'**accuracy**.
- La **précision et le rappel** (precision et *recall* en anglais).

Chaque métrique évalue la performance du modèle avec une approche différente.

Afin d'expliquer ces notions, nous allons introduire 4 termes très importants.

**Arbitrairement**, nous allons choisir que la classe **'republican' sera la classe positive** (1) et **'democrat' sera la classe négative** (0).

Ainsi, nous appellerons :

- **Vrai positif (VP)** une observation classée **positive** ('republican') par le modèle et qui est effectivement **positive** ('republican').
- **Faux positif (FP)** une observation classée **positive** ('republican') par le modèle mais qui était en réalité **négative** ('democrat').
- **Vrai négatif (VN)** une observation classée **négative** ('democrat') par le modèle et qui est effectivement **négative** ('democrat').
- **Faux négatif (FN)** une observation classée **négative** ('democrat') par le modèle mais qui était en réalité **positive** ('republican').

![sklearn_intro_positif_negatif](https://github.com/diaBabPro/colabs/blob/main/sklearn_intro_positif_negatif.png?raw=true)

L'**accuracy** est la métrique la plus couramment utilisée pour évaluer un modèle.
Elle correspond simplement au taux de prédictions **correctes** effectuées par le modèle.

On suppose que l'on dispose de  𝑛
  observations.
On note  VP
  le nombre de Vrais Positifs et  VN
  le nombre de Vrais Négatifs.
L'accuracy est alors donnée par :

$accuracy=\frac{VP+VN}{𝑛}$

La **précision** est une métrique qui répond à la question : **Parmi toutes les prédictions positives du modèle, combien sont de vrais positifs ?**

Si on note  FP
  le nombre de Faux Positifs du modèle, alors la précision est donnée par :

$precision=\frac{VP}{VP+FP}$

Un score de précision élevé nous informe que le modèle ne classe pas aveuglément toutes les observations comme positives.

Le **rappel** est une métrique qui quantifie la proportion d'observations réellement positives qui ont été correctement classifiées positives par le modèle.

Si on note  FN
  le nombre de Faux Négatifs, alors le rappel est donné par :

$rappel=\frac{VP}{VP+FN}$

Un score de rappel élevé nous informe que le modèle est capable de bien détecter les observations réellement positives.

La **matrice de confusion** compte pour un jeu de données les valeurs de VP, VN, FP et FN, ce qui nous permet de calculer les trois métriques précédentes :

$\text{Confusion Matrix} =
\begin{pmatrix}
\text{VN} & \text{FP} \\
\text{FN} & \text{VP}
\end{pmatrix}$

La fonction **`confusion_matrix`** du sous-module `sklearn.metrics` permet de générer la matrice de confusion à partir des prédictions d'un modèle :

```python
confusion_matrix(y_true, y_pred)
```

>- **`y_true`** contient les **vraies** valeurs de y.
>- **`y_pred`** contient les valeurs **prédites** par le modèle.

L'affichage de la matrice de confusion peut se faire aussi avec la fonction **pd.crosstab** :

- **(a)** Importer les fonctions **`accuracy_score`**, **`precision_score`** et **`recall_score`** du sous-module `sklearn.metrics`.
- **(b)** Afficher la matrice de confusion des prédictions du modèle **`logreg`** à l'aide de **`pd.crosstab`**.
- **(c)** Calculer l'accuracy, la précision et le rappel des prédictions du modèle **`logreg`**. Pour utiliser les métriques `precision_score` et `recall_score`, il faudra renseigner l'argument **`pos_label = 'republican'`** afin de préciser que la classe `'republican'` est la classe positive.

In [ ]:
# Insérez votre code ici

## **Recap**
Scikit-learn propose de nombreux modèles de classification comme **`LogisticRegression`**.

L'utilisation de ces modèles se fait de la même façon pour **tous** les modèles de scikit-learn :

- **Instanciation** du modèle.
- **Entraînement** du modèle : **`model.fit(X_train, y_train)`**.
- **Prédiction** : **`model.predict(X_test)`**.

La prédiction sur le jeu de test nous permet d'**évaluer** la performance du modèle grâce à des **métriques** adaptées.

Les métriques que nous avons vues s'utilisent pour la classification **binaire** et se calculent grâce à 4 valeurs :

- Vrais Positifs : Prédiction = + | Réalité = +
- Vrais Négatifs : Prédiction = - | Réalité = -
- Faux Positifs : Prédiction = + | Réalité = -
- Faux Négatifs : Prédiction = - | Réalité = +

Toutes ces valeurs peuvent se calculer à l'aide de la **matrice de confusion** générée par la fonction **`confusion_matrix`** du sous-module `sklearn.metrics` ou par la fonction **`pd.crosstab`**.

Grâce à ces valeurs, nous pouvons calculer des métriques comme :

- L'**accuracy** : La proportion d'observations correctement classifiées.
- La **précision** : La proportion de vrais positifs parmi toutes les prédictions positives du modèle.
- Le **rappel** : La proportion d'observations réellement positives qui ont été correctement classifiées positives par le modèle.

Toutes ces métriques peuvent s'obtenir à l'aide de la fonction **`classification_report`** du sous-module **`sklearn.metrics`**.